In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import einops
import random
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import os
from torchmetrics import StructuralSimilarityIndexMeasure

In [2]:
EPOCHS = 3000
BATCH_SIZE = 256

In [3]:
seed = 42
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [4]:
# 1. ImageNet 데이터셋 불러오기
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # 이미지 크기를 64x64로 조정
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

# ImageNet 데이터 로드
train_data = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform)
# 시드 설정을 위한 생성기(generator) 생성
g = torch.Generator()
g.manual_seed(42)

# DataLoader 설정
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, generator=g)
val_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified


In [5]:
class VAE(nn.Module):
    def __init__(self, image_size=64, latent_dim=128):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )

        self.fc_mu = nn.Linear(512 * 4 * 4, latent_dim)  # 잠재 공간의 평균
        self.fc_logvar = nn.Linear(512 * 4 * 4, latent_dim)  # 로그 분산

        self.fc_decode = nn.Linear(latent_dim, 512 * 4 * 4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()  # 이미지 생성이므로 [-1, 1] 범위로 출력
        )

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)  # 표준편차 계산
        eps = torch.randn_like(std)     # 노이즈 생성
        
        return mu + std * eps 

    def decode(self, z):
        h = self.fc_decode(z)
        h = h.view(-1, 512, 4, 4)
        return self.decoder(h)

    def forward(self, x):
        # torch.Size([128, 3, 64, 64])

        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)

        return self.decode(z), mu, logvar



# VAE 손실 함수 정의
# VAE 학습 함수
def vae_loss_fn(recon_x, x, mu, logvar, criterion):
    #mask = (x != 0).float()  # 입력값에서 0인 값에 대한 마스크 적용

    # 재구성 손실 (MSE) 계산
    recon_loss = criterion(recon_x , x )

    # attention_map이 양수일 때는 KL 손실을 계산하지 않도록 마스크 적용
    #kl_mask = (attention_map <= 0).float()  
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # 총 손실 계산
    return recon_loss + kl_loss



In [6]:
import torch
import torch.nn.functional as F
import torchvision.utils as vutils
import numpy as np
import os
import matplotlib.pyplot as plt

# 랜덤 패치 마스킹 함수
def random_mask(images, mask_size=8, num_patches=16):
    masked_images = images.clone()
    _, _, height, width = images.size()
    seed = 42
    torch.manual_seed(seed)
    for _ in range(num_patches):
        top = np.random.randint(0, height - mask_size)
        left = np.random.randint(0, width - mask_size)
        masked_images[:, :, top:top + mask_size, left:left + mask_size] = 0
    return masked_images

def calculate_psnr(reconstructed, original):
    mse = F.mse_loss(reconstructed, original, reduction='none')
    mse = mse.mean(dim=[1, 2, 3])  # 각 이미지의 MSE
    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))  # PSNR 계산
    return psnr

# Evaluate VAE with MSE and PSNR, and save masked images and attention maps
def evaluate_vae(vae, data_loader, device, save_dir):
    vae.eval()
    total_mse = 0.0
    total_psnr = 0.0
    total_images = 0
    os.makedirs(save_dir, exist_ok=True)

def train(model, train_loader,val_loader, num_epochs,criterion, save_dir, device):
    # 생성된 이미지를 저장할 디렉터리 설정
    
    best_loss = float('inf')
    avg_loss = float('inf')
    losses, psnrs, mses, ssims = [], [], [], []
    best_model = None
    os.makedirs(save_dir, exist_ok=True)
    ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch_idx, (images, _) in enumerate(train_loader):
            images = images.to(device)
            
            # 이미지에 랜덤 마스킹 적용
            masked_images = random_mask(images)
            
            optimizer.zero_grad()
            
            # VAE에서 복원된 이미지와 잠재 벡터를 얻음
            recon_images, mu, logvar = model(masked_images)
            loss = vae_loss_fn(recon_images, images, mu, logvar, criterion)  # 실제 원본 이미지와 비교
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        avg_loss = train_loss/len(train_loader)
        losses.append(avg_loss)

        if avg_loss<best_loss:
            best_model = model.state_dict()
            best_loss = avg_loss
            
        print(f"Epoch {epoch+1}, Loss: {avg_loss}")
        
        # 100 에포크마다 이미지를 복원하고 저장
        if (epoch) % 100 == 0:
            model.eval()
            with torch.no_grad():
                for batch_idx, (images, _) in enumerate(val_loader):
                    images = images.to(device)
                    
                    # 이미지에 랜덤 마스킹 적용
                    masked_images = random_mask(images)
                    
                    optimizer.zero_grad()
                    
                    # VAE에서 복원된 이미지와 잠재 벡터를 얻음
                    recon_images, mu, logvar = model(masked_images)


                    # 원본 배치에서 이미지 가져오기
                    original_images = images.cpu()
                    masked_images_cpu = masked_images.cpu()
        
                    reconstructed_images = recon_images.cpu()
                    reconstructed_images = (reconstructed_images + 1) / 2  # [-1, 1] 범위를 [0, 1]로 변환

                    trans_images = (original_images + 1) / 2  # 동일하게 원본 이미지도 변환
                    
                    # MSE 및 PSNR 계산
                    mse = F.mse_loss(reconstructed_images, trans_images, reduction='sum').item()  # 배치의 MSE
                    psnr = calculate_psnr(reconstructed_images, trans_images).sum().item()  # 배치의 PSNR
                    ssim = ssim_metric(reconstructed_images, trans_images)
                    
                    avg_mses = mse/BATCH_SIZE
                    avg_psnrs = psnr/BATCH_SIZE
                    avg_ssims = ssim
                    
                    mses.append(avg_mses)
                    psnrs.append(avg_psnrs)
                    ssims.append(avg_ssims)
                    
                    print(f"Average MSE per 1 image : {avg_mses}")
                    print(f"Average PSNR per 1 image : {avg_psnrs}")
                    print(f"Average SSIM per 1 image: {avg_ssims}")
                    
        
                    # 이미지 시각화 및 저장
                    fig, axs = plt.subplots(3, 1, figsize=(8, 24))
        
                    # 원본 이미지 시각화
                    axs[0].set_title(f"Original Images at Epoch {epoch+1}")
                    original_grid = vutils.make_grid(original_images, padding=2, normalize=True)
                    axs[0].imshow(np.transpose(original_grid, (1, 2, 0)))
                    axs[0].axis("off")
        
                    # 마스킹된 이미지 시각화
                    axs[1].set_title(f"Masked Images at Epoch {epoch+1}")
                    masked_grid = vutils.make_grid(masked_images_cpu, padding=2, normalize=True)
                    axs[1].imshow(np.transpose(masked_grid, (1, 2, 0)))
                    axs[1].axis("off")
        
                    # 복원된 이미지 시각화
                    axs[2].set_title(f"Reconstructed Images at Epoch {epoch+1}")
                    reconstructed_grid = vutils.make_grid(reconstructed_images, padding=2, normalize=True)
                    axs[2].imshow(np.transpose(reconstructed_grid, (1, 2, 0)))
                    axs[2].axis("off")
        
                    # 이미지 파일로 저장
                    save_path = os.path.join(save_dir, f'VAE_masked_reconstructed_epoch_{epoch+1}.png')
                    plt.savefig(save_path)
                    plt.close()  # plt.show() 대신 close()를 사용해 메모리 관리
                    break
    return losses,mses, psnrs,ssims, best_model

In [7]:
save_dir = './generated_vae'
device = 'cuda'
vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction='sum')  # Mean Squared Error Loss


losses,mses, psnrs,ssims, best_model = train(vae, train_loader,val_loader, EPOCHS,criterion, save_dir, device)

/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


Epoch 1, Loss: 746091.7984375
Average MSE per 1 image : 620.1044311523438
Average PSNR per 1 image : 13.47511100769043
Average SSIM per 1 image: 0.23055651783943176
Epoch 2, Loss: 601892.7125
Epoch 3, Loss: 506731.121875
Epoch 4, Loss: 434525.296875
Epoch 5, Loss: 392781.29296875
Epoch 6, Loss: 367137.534375
Epoch 7, Loss: 343452.90703125
Epoch 8, Loss: 327517.14921875
Epoch 9, Loss: 315534.35546875
Epoch 10, Loss: 301261.34453125
Epoch 11, Loss: 292925.35234375
Epoch 12, Loss: 300694.728125
Epoch 13, Loss: 282167.32265625
Epoch 14, Loss: 264137.378125
Epoch 15, Loss: 250847.44375
Epoch 16, Loss: 243331.86796875
Epoch 17, Loss: 238127.54765625
Epoch 18, Loss: 231458.625
Epoch 19, Loss: 228699.272265625
Epoch 20, Loss: 223572.3171875
Epoch 21, Loss: 217912.234765625
Epoch 22, Loss: 216502.109765625
Epoch 23, Loss: 214171.630859375
Epoch 24, Loss: 209313.08671875
Epoch 25, Loss: 211007.609765625
Epoch 26, Loss: 201935.91875
Epoch 27, Loss: 199805.13515625
Epoch 28, Loss: 196975.533984375

In [8]:
# 에포크별 평균 손실 계산
avg_losses = [epoch_loss / (BATCH_SIZE * len(train_loader)) for epoch_loss in losses]

# 평균 손실값을 확인
print(avg_losses)


[145.7210543823242, 117.55717041015626, 98.97092224121094, 84.86822204589843, 76.71509628295898, 71.70654968261718, 67.08064590454102, 63.96819320678711, 61.627803802490234, 58.840106353759765, 57.211982879638676, 58.72943908691407, 55.11080520629882, 51.58933166503906, 48.99364135742188, 47.52575546264649, 46.50928665161133, 45.2067626953125, 44.667826614379884, 43.666468200683596, 42.56098335266113, 42.28556831359863, 41.83039665222168, 40.88146224975586, 41.21242378234864, 39.44060913085938, 39.02444046020508, 38.47178398132324, 38.02148811340332, 37.98355567932129, 37.368986434936524, 37.0224063873291, 37.03884178161621, 36.90460708618164, 36.266061248779295, 36.1642919921875, 37.37862045288086, 35.95312110900879, 35.305030822753906, 35.069187011718746, 34.72827812194824, 34.315122375488286, 34.307487106323244, 33.94847084045411, 34.15170997619629, 33.82227813720703, 33.41421188354492, 33.20529426574707, 33.257375717163086, 32.6133251953125, 32.48130226135254, 32.469724426269536, 3

In [9]:
def save_metric_to_file(metric, file_name, save_dir, metric_name):
    os.makedirs(save_dir, exist_ok=True)
    file_path = os.path.join(save_dir, file_name)
    
    with open(file_path, 'w') as f:
        f.write(f"{metric_name}:\n")
        for i, value in enumerate(metric, 1):
            f.write(f"{i} {value}\n")
    
    print(f"{metric_name} saved to {file_path}")

In [10]:
# 각각의 메트릭을 별도 파일에 저장
save_metric_to_file(avg_losses, 'vae_epoch_losses.txt', save_dir, 'Loss')
save_metric_to_file(mses, 'vae_mean_squared_errors.txt', save_dir, 'MSE')
save_metric_to_file(psnrs, 'vae_peak_signal_to_noise_ratios.txt', save_dir, 'PSNR')

Loss saved to ./generated_vae/vae_epoch_losses.txt
MSE saved to ./generated_vae/vae_mean_squared_errors.txt
PSNR saved to ./generated_vae/vae_peak_signal_to_noise_ratios.txt


In [11]:
vae.load_state_dict(best_model)

<All keys matched successfully>

In [13]:
prn

NameError: name 'prn' is not defined

In [ ]:
import torch.nn.functional as F
import math
import time
import torchvision.utils as vutils
all_time = []
# PSNR 계산 함수 정의


    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(data_loader):
            images = images.to(device)
            masked_images = random_mask(images)
            
            # VAE로 복원된 이미지 얻기
            now = time.time()
            recon_images, mu, logvar, attention_out1, attention_out3 = vae(masked_images)

            attention_out1 = einops.rearrange(attention_out1, 'b c (h w) -> b c h w', h=64)

            print("Inference time:", time.time() - now)
            recon_images = (recon_images + 1) / 2  # [-1, 1] 범위를 [0, 1]로 변환
            images = (images + 1) / 2  # 동일하게 원본 이미지도 변환

            # MSE 및 PSNR 계산
            mse = F.mse_loss(recon_images, images, reduction='sum').item()  # 배치의 MSE
            psnr = calculate_psnr(recon_images, images).sum().item()  # 배치의 PSNR
            
            total_mse += mse
            total_psnr += psnr
            total_images += images.size(0)

            # 마스킹된 이미지 및 attention map 시각화 및 저장
            masked_images_cpu = masked_images.cpu()
            attention_map_cpu = attention_out1.cpu()  # attention_map을 [B, 1, H, W]로 가정
            
            for i in range(masked_images_cpu.size(0)):
                fig, axs = plt.subplots(1, 2, figsize=(8, 4))
                
                # 마스킹된 이미지 저장
                axs[0].set_title("Masked Image")
                masked_img = masked_images_cpu[i]
                masked_grid = vutils.make_grid(masked_img, normalize=True, scale_each=True)
                axs[0].imshow(np.transpose(masked_grid.numpy(), (1, 2, 0)))
                axs[0].axis("off")

                # Attention map 저장
                axs[1].set_title("Masked Image")
                attention_img = attention_map_cpu[i]
                attention_grid = vutils.make_grid(attention_img, normalize=True, scale_each=True)
                axs[1].imshow(np.transpose(attention_grid.numpy(), (1, 2, 0)))
                axs[1].axis("off")
                break
                # # 이미지 파일로 저장
                # save_path = os.path.join(save_dir, f"batch_{batch_idx}_img_{i}_masked_attention.png")
                # plt.savefig(save_path)
                plt.close()

    avg_mse = total_mse / total_images
    avg_psnr = total_psnr / total_images
    return avg_mse, avg_psnr

In [ ]:
avg_mse, avg_psnr = evaluate_vae(vae, train_loader, device,"./")
print(f"Epoch {epoch+1}, Average MSE: {avg_mse}, Average PSNR: {avg_psnr} dB")